In [20]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
import copy

In [21]:
corpus = pd.read_csv('corpus')

In [22]:
corp = corpus

In [23]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [24]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [25]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [26]:
doc_tokens = corpus['tokens']

In [27]:
dictionary = gensim.corpora.Dictionary(doc_tokens)

In [28]:
corpus = [dictionary.doc2bow(text) for text in doc_tokens]

In [29]:
filtered_dict = copy.deepcopy(dictionary)

In [30]:
filtered_dict.filter_extremes(no_below=30, no_above=0.5)

In [31]:
if len(filtered_dict) > 0:
    dictionary = filtered_dict
    corpus = [dictionary.doc2bow(text) for text in doc_tokens]

In [32]:
# tfidf = gensim.models.TfidfModel(corpus, dictionary=dictionary)
# corpus = tfidf[corpus]

In [33]:
cv_scores = []
umass_scores = []
model_list = {}
max_topic_clusters = 10
metrics = pd.DataFrame()
best_model = None
best_model_clusters = None

In [34]:
for topics in range(10, max_topic_clusters + 1):
    print("Running LDA with {0} topic(s)".format(topics))
    lm = gensim.models.LdaMulticore(corpus= corpus, num_topics=topics, id2word=dictionary, passes=100,
                                    chunksize=2000, alpha='symmetric', eval_every=10,
                                    random_state=42)
    model_list[topics] = lm
    
    cm_1 = gensim.models.CoherenceModel(model=lm, texts=doc_tokens, dictionary=dictionary,coherence='c_v').get_coherence()
    cm_2 = gensim.models.CoherenceModel(model=lm, corpus=corpus, dictionary=dictionary,coherence='u_mass').get_coherence()
    lp = lm.log_perplexity(corpus)
    cv_scores.append(cm_1)
    umass_scores.append(cm_2)

Running LDA with 10 topic(s)


In [35]:
metrics = pd.DataFrame(
            {'topics': range(10, max_topic_clusters + 1),'cv_score': cv_scores,
             'umass_score': umass_scores,'log_perplexity': lp})

In [36]:
metric='cv_score'
n_topics = metrics[metrics[metric] == metrics[metric].max()]['topics'].values[0]

In [37]:
n_topics

10

In [38]:
best_model = model_list.get(n_topics)
best_model_clusters = n_topics

In [39]:
for idx, topic in best_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.387*"market" + 0.056*"perfect" + 0.054*"per" + 0.053*"equilibrium" + 0.033*"procedure" + 0.019*"happens" + 0.016*"minimum" + 0.015*"distinct" + 0.014*"limit" + 0.013*"claim"
Topic: 1 
Words: 0.352*"matrix" + 0.047*"computed" + 0.038*"approximation" + 0.029*"element" + 0.028*"bound" + 0.025*"proposed" + 0.023*"consists" + 0.022*"convergence" + 0.021*"claim" + 0.021*"write"
Topic: 2 
Words: 0.008*"experiment" + 0.008*"gain" + 0.008*"goal" + 0.008*"enough" + 0.008*"equally" + 0.008*"full" + 0.008*"difficult" + 0.008*"generate" + 0.008*"knowledge" + 0.008*"illustrated"
Topic: 3 
Words: 0.287*"degree" + 0.060*"parameter" + 0.046*"subset" + 0.029*"construct" + 0.028*"global" + 0.027*"edges" + 0.026*"bound" + 0.023*"element" + 0.017*"minimum" + 0.016*"cycle"
Topic: 4 
Words: 0.445*"web" + 0.036*"communication" + 0.033*"topic" + 0.030*"paper" + 0.029*"com" + 0.024*"global" + 0.021*"direction" + 0.016*"developed" + 0.016*"mind" + 0.016*"early"
Topic: 5 
Words: 0.179*"alternat